IMPORTING REQUIRED MODULE
-
- 
OpenCV - For image manipulation
- 
Numpy - For matrix manipulation
- 
OS - For file handling and directory operations

In [1]:
import cv2 as cv # cv2 is openCV
import numpy as np
import os

In [23]:
raw_data = "../MP SEM3/Raw Data" # Path of raw data
dataset = "../MP SEM3/Dataset" # Path of dataset where the processed image will be stored
drawing = False
ix,iy = -1,-1

global pic_name, folder
pic_name = 1

## Opening face detection module, Haarcascade file saved as xml ##
face_cascade = cv.CascadeClassifier(r"../MP SEM3/haarcascades/haarcascade_frontalface_default.xml")

User defined functions for Manual croping of Images Using mouse cursor
-

In [38]:
## This function draws the rectangle on the passed image using cursor cordinates ##
def draw_rect(event,x,y,flags,params):
    
    global ix,iy,drawing, pic_name
    
    img2 = image.copy()
    text = '1.Draw a rectangle and press SPACE.'
    cv.putText(image, text, org=(10,25), fontFace=font, fontScale=1.5, color= (0,0,255),thickness=2)
    text = '2.Only press SPACE if MISSFIT.'
    cv.putText(image, text, org=(10,50), fontFace=font, fontScale=1.5, color= (0,0,255),thickness=2)


    if event == cv.EVENT_LBUTTONDOWN:
        
        drawing = True
        ix,iy=x,y
        
    elif event == cv.EVENT_MOUSEMOVE:
        
        if drawing == True:
            global   x1,y1
            x1,y1=x,y
            #cv.rectangle(image,(ix,iy),(x1,y1),(0,255,0),1)
            
    elif event == cv.EVENT_LBUTTONUP:
        drawing = False
        cv.rectangle(image,(ix,iy),(x1,y1),(0,0,255),2)
        croped = img2[iy:y1,ix:x1]
        cv.imwrite(os.path.join(folder,f"{pic_name}.jpg"),croped)
        pic_name = pic_name+1

In [39]:
## This function help us take cursor input directly from the user ##
def mouse(image):
    cv.namedWindow("face_crop") # A window will appear with name "face_crop"


    cv.setMouseCallback("face_crop",draw_rect) # Window named "face_crop" is passed to function draw_rect

    while True:

        cv.imshow('face_crop',image)
        
        key = cv.waitKey(1) & 0xFF
        if key == ord(' '):
            break
    
    cv.destroyAllWindows()
    

ALGORITHM FOR CREATING FINAL DATASET 
-
- 
A nested loop iterates over every image of every folder from 'raw_data'
- 
It creates subfolders in 'dataset' with the same name as in 'raw_data' folder

In [40]:
## First for loop iterates over every folder in the "raw_data" ##
for name in os.listdir(raw_data):
    folder = os.path.join(dataset,name)
    
    ## Creates a subfolder(with same name in raw_data) inside the dataset folder ##
    if name not in os.listdir(dataset):
        os.mkdir(folder)
    pic_name = 1
    
    ## Second loop iterates over every images in the every subfolder of raw_data ##
    for pics in os.listdir(f"{raw_data}/{name}"):
        
        image = cv.imread(f"{raw_data}/{name}/{pics}",1) # Reades image in BGR format
        global img
        img = image.copy() # creating a copy of image
        
        x,y = 0,0
        gimg = cv.cvtColor(image, cv.COLOR_BGR2GRAY) # Converting the image to greyscale
        
        ## Detects the face and return a tuple of co-ordinates and dimension of face ##
        face_rect = face_cascade.detectMultiScale(gimg,scaleFactor = 1.21, minNeighbors = 7)
        
        ## Checking if the detected face is apropriate ##
        if np.shape(face_rect)[0] == 0:
            image = cv.resize(img,(500,500))
            mouse(image)
            continue
        elif len(face_rect)>1:
            image = cv.resize(img,(500,500))
            mouse(image)
            continue
            
        ## Draw a rectangle denoting the detected image ##    
        for (x,y,w,h) in face_rect:
            cv.rectangle(image, (x,y),(x+w,y+h),(0,255,0),10)
            
        image = cv.resize(image,(500,500)) # resize the image to constant dimension 
        
        font = cv.FONT_HERSHEY_PLAIN
        text1 = '1.Click Space for saving.'
        cv.putText(image, text1, org=(10,30), fontFace=font, fontScale=1.5, color= (0,0,255),thickness=2)
        text2 = '2.Click N for manual Cropping.'
        cv.putText(image, text2, org=(10,50), fontFace=font, fontScale=1.5, color= (0,0,255),thickness=2)
        
        cv.imshow('pics',image) # Display image
        
        cropped_image = img[y-1:y+h+1, x-1:x+w] # Cropping out the face from full image
        
        global key
        key = cv.waitKey(0) & 0xFF # taking a input from keyboard as confermation key
        
        if key == ord(' '):
            cv.imwrite(os.path.join(folder,f"{pic_name}.jpg"),cropped_image) # Saving the cropped face in dataset folder
            pic_name = pic_name+1 #name of the image for convenience
        
        elif key == ord('n'):
            image = cv.resize(img,(500,500))
            mouse(image) #calls function for cropping image manually
            continue
        elif key == ord('c'):
            break
cv.destroyAllWindows()